In [1]:
import pandas as pd
import sqlite3

In [2]:
dbase = 'chinook.db'

def run_query(dbase,sql):
    with sqlite3.connect(dbase) as conn:
        return pd.read_sql(sql,conn)
    
def run_command(dbase,sql):
    with sqlite3.connect(dbase) as conn:
        conn.isolation_level = None
        conn.execute(sql)
        
def show_tables(dbase):
    qry= '''
            select
                name,
                type
            from sqlite_master
            where type in("table","view");
    '''
    return run_query(dbase,qry)    

In [3]:
show_tables(dbase)

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


In [4]:
sql = '''
        with usa_sales_by_genre as (
                                select name as genre_name,
                                        count(track_id) as total_tracks_sold
                                from
                                    (
                                      select g.genre_id,
                                            g.name,
                                            t.track_id,
                                            inline.invoice_id,
                                            inv.billing_country
                                    from genre as g
                                    left join track as t
                                    on g.genre_id=t.genre_id
                                    left join invoice_line as inline
                                    on t.track_id=inline.track_id
                                    left join invoice as inv
                                    on inline.invoice_id=inv.invoice_id
                                    where
                                        inv.billing_country='USA')
                                group by genre_name
                                ),
                all_sales_by_genre as (select name as genre_name,
                                        count(track_id) as total_tracks_sold
                                from
                                    (
                                      select g.genre_id,
                                            g.name,
                                            t.track_id,
                                            inline.invoice_id,
                                            inv.billing_country
                                    from genre as g
                                    left join track as t
                                    on g.genre_id=t.genre_id
                                    left join invoice_line as inline
                                    on t.track_id=inline.track_id
                                    left join invoice as inv
                                    on inline.invoice_id=inv.invoice_id)
                                group by genre_name
                                )
        
select  usa.genre_name,
        usa.total_tracks_sold as sales_usa,
        al.total_tracks_sold as sales_all,
        cast(usa.total_tracks_sold as float)/cast(al.total_tracks_sold as float) as percentage
from usa_sales_by_genre as usa
left join all_sales_by_genre as al
on usa.genre_name=al.genre_name
order by 2 desc
limit 10;


'''

run_query(dbase,sql)

,genre_name,sales_usa,sales_all,percentage
0,Rock,561,3017,0.185946
1,Alternative & Punk,130,648,0.200617
2,Metal,124,755,0.164238
3,R&B/Soul,53,165,0.321212
4,Blues,36,149,0.241611
5,Alternative,35,123,0.284553
6,Latin,22,627,0.035088
7,Pop,22,86,0.255814
8,Hip Hop/Rap,20,47,0.425532
9,Jazz,14,190,0.073684


In [5]:
albums_to_purchase = '''
WITH usa_tracks_sold AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name genre,
    count(uts.invoice_line_id) tracks_sold,
    cast(count(uts.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold
    ) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;
'''

run_query(dbase,albums_to_purchase)

,genre,tracks_sold,percentage_sold
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Latin,22,0.020932
7,Pop,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


In [6]:
sql = '''
        select e.employee_id,
                e.first_name || ' ' || e.last_name as emp_fullname,
                e.country,
                e.hire_date,
                e.title,
                sum(inv.total) as sales_amt
        from employee as e
        left join customer as c
        on e.employee_id=c.support_rep_id
        left join invoice as inv
        on c.customer_id=inv.customer_id
        group by 1,2
        order by sales_amt desc;
    
'''
run_query(dbase,sql)


,employee_id,emp_fullname,country,hire_date,title,sales_amt
0,3,Jane Peacock,Canada,2017-04-01 00:00:00,Sales Support Agent,1731.51
1,4,Margaret Park,Canada,2017-05-03 00:00:00,Sales Support Agent,1584.00
2,5,Steve Johnson,Canada,2017-10-17 00:00:00,Sales Support Agent,1393.92
3,1,Andrew Adams,Canada,2016-08-14 00:00:00,General Manager,NaN
4,2,Nancy Edwards,Canada,2016-05-01 00:00:00,Sales Manager,NaN
5,6,Michael Mitchell,Canada,2016-10-17 00:00:00,IT Manager,NaN
6,7,Robert King,Canada,2017-01-02 00:00:00,IT Staff,NaN
7,8,Laura Callahan,Canada,2017-03-04 00:00:00,IT Staff,NaN


In [20]:
sql = '''
        with customer_count as (
                                select country,
                                        customer_count,
                                        case
                                            when country='Other' then 1
                                            else 0
                                            end as sort
                                from(
                                    select country,
                                            sum(customer_count) as customer_count
                                    from (
                                        select case
                                                when customer_count=1 then
                                                "Other"
                                                else 
                                                country
                                                end as country,
                                                customer_count
                                        from (
                                            select country, count(distinct customer_id) as customer_count
                                            from customer
                                            group by country
                                        )
                                    )
                                    group by country
                                    order by customer_count desc
                                    )
                                    order by sort asc
                                )
                                
                    
        select * from customer_count
                            
                                
                                

'''

run_query(dbase,sql)

,country,customer_count,sort
0,USA,13,0
1,Canada,8,0
2,Brazil,5,0
3,France,5,0
4,Germany,4,0
5,United Kingdom,3,0
6,Czech Republic,2,0
7,India,2,0
8,Portugal,2,0
9,Other,15,1


In [37]:
sql = '''
        with customer_count as (
                                select country, count(distinct customer_id) as customer_count
                                from customer
                                group by country
                                ),
            total_sales as (
                            select distinct c.country,
                                    amt.sales_amt
                            from customer as c
                            left join (select billing_country,
                                                sum(total) as sales_amt
                                        from invoice
                                        group by billing_country) as amt
                            on c.country=amt.billing_country
                                        ),
            total_orders as (
                            select i.billing_country as country,
                                    count(distinct il.invoice_id) as order_count
                            from invoice as i
                            left join invoice_line as il
                            on i.invoice_id = il.invoice_id
                            group by i.billing_country
                            )
                                    
                                
        
        select country,
                customer_count,
                sales_amt,
                sales_amt/customer_count as avg_sales_per_cust,
                sales_amt/order_count as avg_order_value
        from (
        
            select country,
                    sum(customer_count) as customer_count,
                    sum(sales_amt) as sales_amt,
                    sum(order_count) as order_count
            from (
                select case
                        when customer_count=1 then "Other"
                        else country
                        end as country,
                        customer_count,
                        sales_amt,
                        order_count
                from (
                    select c.*,
                            s.sales_amt,
                            o.order_count
                    from customer_count as c
                    left join total_sales as s
                    on c.country=s.country
                    left join total_orders as o
                    on s.country=o.country
                )
            )
            group by country
        )
                            
                                
                                

'''

run_query(dbase,sql)

,country,customer_count,sales_amt,avg_sales_per_cust,avg_order_value
0,Brazil,5,427.68,85.536000,7.011148
1,Canada,8,535.59,66.948750,7.047237
2,Czech Republic,2,273.24,136.620000,9.108000
3,France,5,389.07,77.814000,7.781400
4,Germany,4,334.62,83.655000,8.161463
5,India,2,183.15,91.575000,8.721429
6,Other,15,1094.94,72.996000,7.448571
7,Portugal,2,185.13,92.565000,6.383793
8,USA,13,1040.49,80.037692,7.942672
9,United Kingdom,3,245.52,81.840000,8.768571


In [39]:
sql = '''
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;
'''

run_query(dbase,sql)

,album_purchase,number_of_invoices,percent
0,no,500,0.814332
1,yes,114,0.185668
